In [2]:
import jsonlines
import numpy as np
import cairo
import matplotlib 
from PIL import Image
%matplotlib notebook

In [15]:
file = open('full_simplified_candle.ndjson',encoding='utf-8')

In [16]:
allScratches = jsonlines.Reader(file)

In [17]:
def add_points(obj):
    scratch = np.zeros(( 256, 512))
    for j in obj['drawing']:
        for k in range(len(j[0])):
            x = j[0][k] + 256
            y = j[1][k] 
            scratch[y][x]  = 255
    return scratch

def paint_lines(obj, line_width = 3 ):
    
    line_diameter=60 #bylo 20 
    surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, 256, 256)
    cr = cairo.Context(surface)
    cr.set_antialias(cairo.ANTIALIAS_BEST)
    cr.set_line_cap(cairo.LINE_CAP_ROUND)
    cr.set_line_join(cairo.LINE_JOIN_ROUND)
    cr.set_line_width(line_diameter)
    cr.set_line_width(line_width) #bylo 3

    for j in obj['drawing']:
        cr.set_source_rgb(0,0,0)
        cr.paint()
        for k in range(0, len(j[0])-1):
            cr.set_source_rgb(255, 255, 255)
            x = j[0][k]
            y = j[1][k]
            cr.move_to(x, y)
            x1 =  j[0][(k+1)]
            y1 =  j[1][(k+1)]
            cr.line_to(x1, y1 )
    cr.stroke()          
    
    data = surface.get_data()
    
    newscratch = np.copy(np.asarray(data)[::4])
    
    lines = newscratch.reshape(256, 256)
    
    return lines

def merge_dots_and_lines(scratch, lines):
    scratch[  :  ,  : 256 ] = lines
    scratch = scratch / 255
    final = np.where(scratch < 0.1, 1 , 0 )
    
    return final

def save_training_img(final, name):
    matplotlib.image.imsave(name, final,cmap='gray')
    
    
def mask_to_13(lines):
    
    scratch = np.zeros(( 256, 512))
    
    a = np.zeros((256, 256))
    
    for x in range(0, 256, 1):
        if  x % 19 == 0:
            for y in range(0, 256, 1):   
                if  y  % 19 == 0:
                    a[x][y] = 1
                    
                    
    mask_13   = (a > 0)
    mask_lin  = ( lines > 0)
    common = np.logical_and(mask_13 , mask_lin)
    common = common * 250
    
    scratch[  :  , 256 : 512 ] = common
    
    return scratch
    
    

In [18]:
for obj in allScratches:
    dots  = add_points(obj)
    lines = paint_lines(obj, 18)
    lines_i = paint_lines(obj, 3)
    masked = mask_to_13(lines)
    final = merge_dots_and_lines(masked, lines_i)
    image_name = obj['key_id'] + '.jpg'
    save_training_img(final, image_name)
    print(image_name)

5871980177784832.jpg
5858136034377728.jpg
6377906869633024.jpg
6413533405249536.jpg
6509722804944896.jpg
6548264633499648.jpg
6119740277784576.jpg
5073953611579392.jpg
4808552981987328.jpg
5158168390795264.jpg
6111097885032448.jpg
4552351840993280.jpg
5577437926653952.jpg
4599121690230784.jpg
6657840615784448.jpg
5596280552161280.jpg
6280425829826560.jpg
5314149020073984.jpg
4550003840253952.jpg
6235739291385856.jpg
5929408097419264.jpg
5949724282585088.jpg
6608770270494720.jpg
6609325512458240.jpg
5478206062723072.jpg
6607405515276288.jpg
5706424199938048.jpg
6093001174548480.jpg
4693831888928768.jpg
4765254795919360.jpg
6474769757110272.jpg
5767480079286272.jpg
5232058718748672.jpg
5871054176124928.jpg
4562969352470528.jpg
5013914263748608.jpg
4555165027794944.jpg
4702500491886592.jpg
6552443653455872.jpg
5520848913956864.jpg
5173592553684992.jpg
6119816060469248.jpg
5002150897778688.jpg
4669515398905856.jpg
5900435242090496.jpg
5739142254166016.jpg
4617696530399232.jpg
4859239602323

KeyboardInterrupt: 